In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

In [4]:
from hdna import * 
import nupack as nu
import networkx as nx 
import cProfile, pstats

In [5]:
model = Model('dna', '3D', stacking='nostacking')
s1 = Strand(model, "AAAAAAAAAAAAAAAA")
s2 = s1.complementary()
C = Chamber(model, s1, s2)
N = Kinetwork(model, s1, s2, Geometry(120,270), clean=True)

In [6]:
N.zippingraph()
N.slidingraph()
N.completegraph()

In [7]:
import networkx as nx 
from itertools import pairwise


def get_i(lst):
    indices = []
    for i, (el1, el2) in enumerate(pairwise(lst)):
        if el1 == '.' and el2 != '.':
            indices.append(i)
        if el1 != '.' and el2 == '.':
            indices.append(i+1)
    return indices
#updates current the given structure
def update_structure(string, side, character: str):
    updated = string
    moveslist = []
    moves = get_i(string)[::side]
    for m in moves:
        up = updated[:m] + character + updated[m+1:]
        moveslist.append(up)
    return moveslist
#recursively add nodes and edges to their leafs in a tree like manner until duplexation
#duplexation is automatically recognised by the production of an empty moveslist list 
def leafs(left, right, graph):
    lmoves = update_structure(left, 1, '(')
    rmoves = update_structure(right, -1, ')')
    inbound = '+'.join([left, right])
    zobj = Zippo(model, s1, s2, state='zipping', structure=inbound, clean = True)
    graph.add_node( inbound,
                    obj = zobj,
                    state = zobj.state,
                    pairs = zobj.total_nucleations)
    for lmove, rmove in zip(lmoves, rmoves):
        outbound = '+'.join([lmove,rmove])
        zobj = Zippo(model, s1, s2, state='zipping', structure=outbound, clean = True)
        graph.add_node(outbound,
                        obj = zobj,
                        state = zobj.state,
                        pairs = zobj.total_nucleations)
        graph.add_edge(inbound, outbound) #TODO ADD RATES
        graph.add_edge(outbound, inbound) #TODO ADD RATES
        yield list(leafs(lmove,rmove,graph))
#takes inputted oncore and splits it to update structure simultaneously on the left and the right
left, right = '....(.......+.......)....'.split('+')
subZG = nx.DiGraph()
list(leafs(left, right, subZG))

[[[[[[[[[[[[]]]]]]]],
    [[[[[[[[]]]]]]],
     [[[[[[[]]]]]], [[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]]]]],
   [[[[[[[[[]]]]]]],
     [[[[[[[]]]]]], [[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]]]],
    [[[[[[[[]]]]]], [[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]]],
     [[[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]],
      [[[[[[]]]], [[[[]]], [[[]], [[]]]]],
       [[[[[]]], [[[]], [[]]]], [[[[]], [[]]], [[[]]]]]]]]]],
  [[[[[[[[[[]]]]]]],
     [[[[[[[]]]]]], [[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]]]],
    [[[[[[[[]]]]]], [[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]]],
     [[[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]],
      [[[[[[]]]], [[[[]]], [[[]], [[]]]]],
       [[[[[]]], [[[]], [[]]]], [[[[]], [[]]], [[[]]]]]]]]],
   [[[[[[[[[]]]]]], [[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]]],
     [[[[[[[]]]]], [[[[[]]]], [[[[]]], [[[]], [[]]]]]],
      [[[[[[]]]], [[[[]]], [[[]], [[]]]]],
       [[[[[]]], [[[]], [[]]]], [[[[]], [[]]], [[[]]]]]]]],
    [

In [81]:
def get_i(lst):
    indices = []
    for i, (el1, el2) in enumerate(pairwise(lst)):
        if el1 == '.' and el2 != '.':
            indices.append(i)
        if el1 != '.' and el2 == '.':
            indices.append(i+1)
    return indices
#updates current the given structure
def update_structure(string, side, character: str):
    updated = string
    moveslist = []
    moves = get_i(string)[::side]
    for m in moves:
        up = updated[:m] + character + updated[m+1:]
        moveslist.append(up)
    return moveslist
#recursively add nodes and edges to their leafs in a tree like manner until duplexation
#duplexation is automatically recognised by the production of an empty moveslist list 
def leafs(inbound):
    for inb in inbound:
        left, right = inb.split('+')
        lmoves = update_structure(left, 1, '(')
        rmoves = update_structure(right, -1, ')')
        # print('in   ',inb)
        outbound = []
        for lmove, rmove in zip(lmoves, rmoves):
            outbound.append('+'.join([lmove, rmove]))
        return outbound

def tree(structure):
    for leaf in leafs([*structure]):
        # print(tree(leaf))
        return tree(leafs([*leaf]))
        # yield list(leafs(lmove,rmove))
#takes inputted oncore and splits it to update structure simultaneously on the left and the right
left, right = '....(.......+.......)....'.split('+')
structure = '....(.......+.......)....'
tree(structure)

ValueError: not enough values to unpack (expected 2, got 1)